In [1]:
import requests
import json
import pandas as pd
from pandas import json_normalize
import config
from db_manager import DBManager
import os
from rest_api import Api
from datetime import date, datetime
from dateutil.relativedelta import relativedelta
from urllib.parse import quote_plus
from sqlalchemy import create_engine
import itertools
import logging
import pickle
import hashlib
import gspread

c:\Users\afroj\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
c:\Users\afroj\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:62: UserWarning: Pandas requires version '1.3.4' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


In [2]:
logging.basicConfig(
    format='%(levelname)s-%(lineno)s-%(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
    level='INFO'
)
logger=logging.getLogger(__name__)

#### Show all columns without truncating

Today's date

In [3]:
today = date.today()
logger.info(f'Current date: {today}')

INFO-__main__:2-Current date: 2023-11-10


### Reading Gsheet to DF

In [4]:
# Authenticate with Google Sheets
gc = gspread.service_account(filename='service_account.json')

# Open the Google Sheets spreadsheet
worksheet = gc.open('Indian Holiday List').sheet1

# Read data from Google Sheets
data = worksheet.get_all_records()

# Convert the data into a Pandas DataFrame
gsheet_df = pd.DataFrame(data)

In [5]:
credentials=config.pg_credentials
db = DBManager(credentials=credentials)
db.run_etl_query(query_file_name='truncate_indian_festivals')
db.create_table_from_df(df=gsheet_df,table_name='indian_festivals')

INFO-db_manager:29-CONNECTED
INFO-db_manager:68-Running DDL Query truncate_indian_festivals
INFO-db_manager:74-Execution Completed
INFO-db_manager:90-Dumped to DB


Adding months to today's date. Near term travel dates 30 - 60 days ahead

In [32]:
# departure_date_1_month = today + relativedelta(months=1)
# departure_date_2_month = today + relativedelta(months=2)
# return_date_1_month = departure_date_2_month
# return_date_2_month = today + relativedelta(months=3)
# logger.info(f'Departure_dates:{departure_date_1_month} and {departure_date_2_month} \nReturn dates:{return_date_1_month} and {return_date_2_month}')

Get the nearby festival departure dates by start_date from the public.indian_holidays table

In [6]:
credentials=config.pg_credentials
db = DBManager(credentials=credentials)
df_festival_near = db.run_query(query_file_name='festival_dates_near')

INFO-db_manager:29-CONNECTED
INFO-db_manager:50-Running Query festival_dates_near
INFO-db_manager:52-Execution Completed


In [7]:
#appending to a list that will contain departure and arrival dates for festivals that are near and  far
credentials=config.pg_credentials
db = DBManager(credentials=credentials)
df_festival_far = db.run_query(query_file_name='festival_dates_far')

INFO-db_manager:29-CONNECTED
INFO-db_manager:50-Running Query festival_dates_far
INFO-db_manager:52-Execution Completed


In [8]:
df_departure_arrival_dates = pd.concat([df_festival_near, df_festival_far], axis=0, ignore_index=True)

In [9]:
df_departure_arrival_dates

,festival,departure_dates,arrival_dates
0,Christmas,2023-12-18,2024-01-18
1,New Years,2023-12-23,2024-01-23
2,Pongal,2024-01-03,2024-02-03


In [10]:
#Converting departure dates to a list
departure_dates=names = df_departure_arrival_dates['departure_dates'].tolist()

In [11]:
departure_dates

[datetime.date(2023, 12, 18),
 datetime.date(2023, 12, 23),
 datetime.date(2024, 1, 3)]

Apply the combination of params to get data for all possible combinations of depart, arrival dates and depart arrival airports

#### Travel payouts API to get flight search results for each combination in the list 

--------------------------------------------------------

API test for amadeus

POST request to the Amadeus authorization server to get the access token

In [12]:
response = requests.post(url=config.url_token, headers=config.headers_token, data=config.data)

if response.status_code == 200:
  # API call succeeded
  token = response.json()['access_token'] 
  logger.info('Successful access token generation')
else:
  # API call failed
  print(response.text)

INFO-__main__:6-Successful access token generation


In [13]:
#Combinations of destination and origin airport codes
origins = config.params['originLocationCode']
destinations = config.params['destinationLocationCode']
departure_dates=departure_dates
# return_dates=[return_date_1_month,return_date_2_month]

Creating combinations of Origin and Destination to be passed as parameters to the api call

trial

In [14]:
headers = {
  'client_id': config.client_id, 
  'client_secret': config.client_secret,
  'Authorization': f'Bearer {token}'
  }

url = config.url

response = []
counter = 0

for origin, dest, depart in itertools.product(origins, destinations, departure_dates):

    params={
        'originLocationCode': origin,
        'destinationLocationCode':dest, 
        'departureDate': depart, 
        'returnDate' : None,
        'adults':1,
        'children':None,
        'infants':None,
        'travelClass':None,
        'currencyCode':'EUR',
        'maxPrice' : None
        }


    #to be worked on later for by calling rest_api module
    # api = Api()
    # resp = api.make_flight_api_request(url,headers,params)
    resp = requests.get(url, headers=headers, params=params)
    counter = counter+1
    resp = resp.json()
    response.append(resp)
    logger.info(f'API call {counter} succeeded for flight offers')
    # else:
    #     # API call failed
    #     logger.info('API call failed')


INFO-__main__:35-API call 1 succeeded for flight offers
INFO-__main__:35-API call 2 succeeded for flight offers
INFO-__main__:35-API call 3 succeeded for flight offers
INFO-__main__:35-API call 4 succeeded for flight offers
INFO-__main__:35-API call 5 succeeded for flight offers
INFO-__main__:35-API call 6 succeeded for flight offers
INFO-__main__:35-API call 7 succeeded for flight offers
INFO-__main__:35-API call 8 succeeded for flight offers
INFO-__main__:35-API call 9 succeeded for flight offers
INFO-__main__:35-API call 10 succeeded for flight offers
INFO-__main__:35-API call 11 succeeded for flight offers
INFO-__main__:35-API call 12 succeeded for flight offers
INFO-__main__:35-API call 13 succeeded for flight offers
INFO-__main__:35-API call 14 succeeded for flight offers
INFO-__main__:35-API call 15 succeeded for flight offers
INFO-__main__:35-API call 16 succeeded for flight offers
INFO-__main__:35-API call 17 succeeded for flight offers
INFO-__main__:35-API call 18 succeeded f

### Saving the Global variable to a file

### Creating the payload for real time price script

In [43]:

# payload = {
#   "data": {
#     "type": "flight-offers-pricing",
#     "flightOffers": [], 
#     "travelers": []
#   }
# }

# for r in response:

#   # Extract flight offer data
#   offers = r['data']
  
#   # Add offers to payload
#   payload['data']['flightOffers'].extend(offers) 

# # Convert payload to JSON string for request 
# json_payload = json.dumps(payload)

In [44]:
# flight_search_response =json_payload

In [45]:
# Save the variable to a file
# with open('global_variable.pkl', 'wb') as file:
#     pickle.dump(flight_search_response, file)

Convert that sample nested JSON data into separate DataFrames

Repeat the same above procedure for itineraries and traveler_pricing

### Offers

In [16]:
offer_df = pd.DataFrame()

for i in range(counter):
    # Assuming response[i]['data'] is a dictionary
    data_to_append = pd.json_normalize(response[i]['data'], record_path=['itineraries'], meta=['id', 'source'])
    
    # Append the data to the existing DataFrame
    offer_df = pd.concat([offer_df, data_to_append], ignore_index=True)

Checking what the offer df contains

To get the first dept airport and last arrival airport looping through each row in itineraries df

In [17]:
offer_df['first_dept_airport'] = offer_df['segments'].apply(lambda x: x[0]['departure']['iataCode']) 
offer_df['last_arr_airport'] = offer_df['segments'].apply(lambda x: x[-1]['arrival']['iataCode'])
offer_df['first_dept_time'] = offer_df['segments'].apply(lambda x: x[0]['departure']['at']) 
offer_df['last_arr_time'] = offer_df['segments'].apply(lambda x: x[-1]['arrival']['at'])

Creating a composite key

Think About a unique key that you will use to join pricing table and segments table together to get the price of the flight

In [18]:
offer_df = offer_df.rename(columns={'id': 'offer_id'})

Trying another unique id

In [19]:
def generate_unique_id(row):
    unique_string = f"{row['offer_id']}_{row['first_dept_airport']}_{row['last_arr_airport']}_{row['first_dept_time']}_{row['last_arr_time']}"
    unique_id = hashlib.md5(unique_string.encode()).hexdigest()
    return unique_id

offer_df['unique_id'] = offer_df.apply(generate_unique_id, axis=1)

In [20]:
offer_df=offer_df.drop(columns=['segments'],axis=1)

In [21]:
# Define the desired column order
desired_order = ['unique_id', 'offer_id', 'source','first_dept_airport','last_arr_airport','first_dept_time','last_arr_time','duration']

# Create a new DataFrame with columns in the desired order
offer_df = offer_df[desired_order]

In [22]:
# converting to datetime type
offer_df['first_dept_time'] = pd.to_datetime(offer_df['first_dept_time'])
offer_df['last_arr_time'] = pd.to_datetime(offer_df['last_arr_time'])

In [23]:
#creating the festival flag column that differentiates between festival dates and near future dates 
offer_df['festival_flag']=1
# Merge the DataFrames based on the date component of "first_dept_time" and "departure_dates"
offer_df['festival'] = offer_df['first_dept_time'].dt.date.map(df_departure_arrival_dates.set_index('departure_dates')['festival'])


In [24]:
now = datetime.now()
offer_df['incremental_day']=now
offer_df['incremental_day'] = pd.to_datetime(offer_df['incremental_day'])

### Itineraries segments

In [58]:
itineraries_df = pd.DataFrame()

for i in range(counter):
    # Assuming response[i]['data'] is a dictionary
    data_to_append = pd.json_normalize(response[i]['data'], record_path=['itineraries'], meta=['id', 'source'])
    
    # Append the data to the existing DataFrame
    itineraries_df = pd.concat([itineraries_df, data_to_append], ignore_index=True)

In [59]:
itineraries_df['first_dept_airport'] = itineraries_df['segments'].apply(lambda x: x[0]['departure']['iataCode']) 
itineraries_df['last_arr_airport'] = itineraries_df['segments'].apply(lambda x: x[-1]['arrival']['iataCode'])
itineraries_df['first_dept_time'] = itineraries_df['segments'].apply(lambda x: x[0]['departure']['at']) 
itineraries_df['last_arr_time'] = itineraries_df['segments'].apply(lambda x: x[-1]['arrival']['at'])

Trying to genearte unique id before exploding

In [60]:
def generate_unique_id(row):
    unique_string = f"{row['id']}_{row['first_dept_airport']}_{row['last_arr_airport']}_{row['first_dept_time']}_{row['last_arr_time']}"
    unique_id = hashlib.md5(unique_string.encode()).hexdigest()
    return unique_id

itineraries_df['unique_id_fk'] = itineraries_df.apply(generate_unique_id, axis=1)

Treating the itineraries df

Just taking the exploded df and carefully scanning each row in the top 5 rows to see what kind of values does the exploded version of offer df contains
exploded df
df2
df

In [61]:
# Explode the 'segment' column to create separate rows for each dictionary
exploded_df = itineraries_df.explode('segments')

# Split the dictionary into columns  
df2 = exploded_df['segments'].apply(pd.Series)

# Merge the new columns back to the original DataFrame
df = exploded_df.merge(df2, right_index=True, left_index=True, suffixes=('_original', '_segments'))

# Drop the original dictionary column
df = df.drop(columns=['segments']) 
df = df.rename(columns={'duration_original': 'duration_total','id_original':'offer_id'})

In [62]:
# Extract departure keys using apply
df['dept_airport'] = df['departure'].apply(lambda x: x['iataCode'])
df['dept_at'] = df['departure'].apply(lambda x: x['at'])
df['arrival_airport'] = df['arrival'].apply(lambda x: x['iataCode'])
df['arrival_at'] = df['arrival'].apply(lambda x: x['at'])
df['aircraft'] = df['aircraft'].apply(lambda x: x['code'])
df.head()
# Drop stops column if it exists
column_to_drop = 'stops'
if column_to_drop in df.columns:
    df.drop(column_to_drop,axis=1,inplace=True)
itineraries_df = df.drop(columns=['departure','arrival','operating','aircraft','blacklistedInEU','first_dept_airport','last_arr_airport','first_dept_time','last_arr_time'])

itineraries_df = itineraries_df.drop_duplicates()
# Reset index
itineraries_df = itineraries_df.reset_index(drop=True)
itineraries_df.head()

,duration_total,offer_id,source,unique_id_fk,carrierCode,number,duration_segments,id_segments,numberOfStops,dept_airport,dept_at,arrival_airport,arrival_at
0,PT10H35M,1,GDS,5708d1cd35002f3efb2a1bd6c4af9b1a,KL,1824,PT1H25M,187,0,BER,2024-01-03T11:50:00,AMS,2024-01-03T13:15:00
1,PT10H35M,1,GDS,5708d1cd35002f3efb2a1bd6c4af9b1a,KL,109,PT8H15M,188,0,AMS,2024-01-03T14:10:00,DEL,2024-01-04T01:55:00
2,PT12H15M,2,GDS,989d90ffd0528dce28571e04905b4384,KL,1822,PT1H25M,117,0,BER,2024-01-03T10:10:00,AMS,2024-01-03T11:35:00
3,PT12H15M,2,GDS,989d90ffd0528dce28571e04905b4384,KL,109,PT8H15M,118,0,AMS,2024-01-03T14:10:00,DEL,2024-01-04T01:55:00
4,PT16H25M,3,GDS,60eed3504368a77c80679e9835624ec4,KL,1818,PT1H25M,153,0,BER,2024-01-03T06:00:00,AMS,2024-01-03T07:25:00


In [63]:
itineraries_df['segment_key'] = itineraries_df['unique_id_fk'].astype(str) + '_' + itineraries_df['id_segments'].astype(str)

Create unique id for segments df

Renaming, reordering and changing datatypes

In [64]:
# Rename the columns according to the SQL table schema
itineraries_df = itineraries_df.rename(columns={
    'source': 'source',
    'duration_segments': 'duration_segments',
    'duration_total': 'duration_total',
    'carrierCode': 'carrier_code',
    'number': 'number',
    'duration_segments': 'duration_segments',
    'numberOfStops': 'number_of_stops',
    'dept_airport': 'dept_airport',
    'dept_at': 'dept_at',
    'arrival_airport': 'arrival_airport',
    'arrival_at': 'arrival_at'
})

# Ensure data types are set correctly
itineraries_df['offer_id'] = itineraries_df['offer_id'].astype(int)
itineraries_df['number_of_stops'] = itineraries_df['number_of_stops'].astype(int)
itineraries_df['dept_at'] = pd.to_datetime(itineraries_df['dept_at'])
itineraries_df['arrival_at'] = pd.to_datetime(itineraries_df['arrival_at'])

now = datetime.now()
itineraries_df['incremental_day']=now
itineraries_df['incremental_day'] = pd.to_datetime(itineraries_df['incremental_day'])


In [65]:
logger.info(f'Current date: {now}')

INFO-__main__:1-Current date: 2023-11-10 08:33:44.026936


In [66]:
# Assuming itineraries_df is your DataFrame
new_column_order = [
    'segment_key',
    'duration_total',
    'offer_id',
    'source',
    'unique_id_fk',
    'carrier_code',
    'number',
    'duration_segments',
    'id_segments',
    'number_of_stops',
    'dept_airport',
    'dept_at',
    'arrival_airport',
    'arrival_at',
    'incremental_day'
]

# Reorder the DataFrame columns
itineraries_df = itineraries_df[new_column_order]


In [67]:
itineraries_df.head()

,segment_key,duration_total,offer_id,source,unique_id_fk,carrier_code,number,duration_segments,id_segments,number_of_stops,dept_airport,dept_at,arrival_airport,arrival_at,incremental_day
0,5708d1cd35002f3efb2a1bd6c4af9b1a_187,PT10H35M,1,GDS,5708d1cd35002f3efb2a1bd6c4af9b1a,KL,1824,PT1H25M,187,0,BER,2024-01-03 11:50:00,AMS,2024-01-03 13:15:00,2023-11-10 08:33:44.026936
1,5708d1cd35002f3efb2a1bd6c4af9b1a_188,PT10H35M,1,GDS,5708d1cd35002f3efb2a1bd6c4af9b1a,KL,109,PT8H15M,188,0,AMS,2024-01-03 14:10:00,DEL,2024-01-04 01:55:00,2023-11-10 08:33:44.026936
2,989d90ffd0528dce28571e04905b4384_117,PT12H15M,2,GDS,989d90ffd0528dce28571e04905b4384,KL,1822,PT1H25M,117,0,BER,2024-01-03 10:10:00,AMS,2024-01-03 11:35:00,2023-11-10 08:33:44.026936
3,989d90ffd0528dce28571e04905b4384_118,PT12H15M,2,GDS,989d90ffd0528dce28571e04905b4384,KL,109,PT8H15M,118,0,AMS,2024-01-03 14:10:00,DEL,2024-01-04 01:55:00,2023-11-10 08:33:44.026936
4,60eed3504368a77c80679e9835624ec4_153,PT16H25M,3,GDS,60eed3504368a77c80679e9835624ec4,KL,1818,PT1H25M,153,0,BER,2024-01-03 06:00:00,AMS,2024-01-03 07:25:00,2023-11-10 08:33:44.026936


Segment key unique check

### Pricing flights

In [69]:
offer = pd.DataFrame()

for i in range(counter):
    # Assuming response[i]['data'] is a dictionary
    data_to_append = pd.json_normalize(response[i]['data'], meta=['id', 'source'])
    
    # Append the data to the existing DataFrame
    offer = pd.concat([offer, data_to_append], ignore_index=True)

In [71]:
pricing_df = pd.DataFrame()

for i in range(counter):
    # Assuming response[i]['data'] is a dictionary
    data_to_append = pd.json_normalize(response[i]['data'], meta=['id', 'source'])
    
    # Append the data to the existing DataFrame
    pricing_df = pd.concat([pricing_df, data_to_append], ignore_index=True)

In [72]:
pricing_df = pricing_df.drop(columns=['instantTicketingRequired','nonHomogeneous','oneWay','lastTicketingDate','lastTicketingDateTime','numberOfBookableSeats','validatingAirlineCodes','travelerPricings'],axis=1)


In [73]:
# Extract first departure airport
pricing_df['first_dept_airport'] = pricing_df['itineraries'].apply(lambda x: x[0]['segments'][0]['departure']['iataCode'])

# Extract last arrival airport
pricing_df['last_arr_airport'] = pricing_df['itineraries'].apply(lambda x: x[0]['segments'][-1]['arrival']['iataCode'])

# Extract first departure time
pricing_df['first_dept_time'] = pricing_df['itineraries'].apply(lambda x: x[0]['segments'][0]['departure']['at'])

# Extract last arrival time
pricing_df['last_arr_time'] = pricing_df['itineraries'].apply(lambda x: x[0]['segments'][-1]['arrival']['at'])

# Drop the 'itineraries' column if no longer needed
pricing_df.drop(columns=['itineraries'], inplace=True)

In [74]:
def generate_unique_id(row):
    unique_string = f"{row['id']}_{row['first_dept_airport']}_{row['last_arr_airport']}_{row['first_dept_time']}_{row['last_arr_time']}"
    unique_id = hashlib.md5(unique_string.encode()).hexdigest()
    return unique_id

pricing_df['unique_id_fk'] = pricing_df.apply(generate_unique_id, axis=1)

#### Remove "price" and dots from column names


In [75]:
pricing_df = pricing_df.rename(columns=lambda x: x.replace('price.', '').replace('price', ''))
pricing_df = pricing_df.rename(columns=lambda x: x.replace('pricingOptions.', '').replace('price', ''))

In [76]:
# Split 'additionalServices' into two columns
pricing_df = pricing_df.drop(['additionalServices','fees','type','id','source','first_dept_time','last_arr_time','last_arr_airport','first_dept_airport'], axis=1)

In [77]:

pricing_df['incremental_day'] =now
pricing_df['incremental_day'] = pd.to_datetime(pricing_df['incremental_day'])


In [78]:
pricing_df.rename(columns={'grandTotal': 'grand_total','fareType': 'fare_type','includedCheckedBagsOnly': 'included_checkedbags_only'}, inplace=True)

In [79]:
pricing_df.head()

,currency,total,base,grand_total,fare_type,included_checkedbags_only,unique_id_fk,incremental_day
0,EUR,479.48,259.00,479.48,[PUBLISHED],False,5708d1cd35002f3efb2a1bd6c4af9b1a,2023-11-10 08:33:44.026936
1,EUR,479.48,259.00,479.48,[PUBLISHED],False,989d90ffd0528dce28571e04905b4384,2023-11-10 08:33:44.026936
2,EUR,479.48,259.00,479.48,[PUBLISHED],False,60eed3504368a77c80679e9835624ec4,2023-11-10 08:33:44.026936
3,EUR,479.48,259.00,479.48,[PUBLISHED],False,36c7ec92630eecd5469b28e6135bc743,2023-11-10 08:33:44.026936
4,EUR,479.48,259.00,479.48,[PUBLISHED],False,1af4088dceef67051970516419ee4507,2023-11-10 08:33:44.026936


In [80]:
credentials=config.pg_credentials
db = DBManager(credentials=credentials)
db.create_table_from_df(df=offer_df,table_name='flight_offers')
db.create_table_from_df(df=itineraries_df,table_name='itineraries')
db.create_table_from_df(df=pricing_df,table_name='pricing')

INFO-db_manager:29-CONNECTED
INFO-db_manager:90-Dumped to DB
INFO-db_manager:90-Dumped to DB
INFO-db_manager:90-Dumped to DB
